Define imports

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import requests
import json
import pandas as pd
pd.options.plotting.backend = 'plotly'

# Call the enpoint to receive all the classes in the event

Swapcard exposes a GraphQL endpoint with information about the event and its classes

In [2]:

url = "https://api.swapcard.com/graphql"

#First payload, around 21 classes
payload="{\"query\":\"{Core_plannings(_eventId:\\\"RXZlbnRfOTQwNDk4\\\"){nodes{categories{name}, bannerUrl,type,fields{__typename,...on Core_MultipleSelectField{ name, values{text}}} withEvent(eventId:\\\"RXZlbnRfOTQwNDk4\\\"){htmlDescription,totalAttendees,beginsAt, title}}}}\",\"variables\":{}}"
headers = {
  'Content-Type': 'application/json; charset=utf-8'
}

response = requests.request("POST", url, headers=headers, data=payload)

resp = json.loads(response.text)


data = resp["data"]["Core_plannings"]["nodes"]


In [3]:
print(data[0])

{'categories': [{'name': 'Additive Manufacturing'}, {'name': 'Machining'}, {'name': 'Sustainability'}], 'bannerUrl': 'https://cdn-api.swapcard.com/public/images/4ecd7118d34b48fda324e416258327d2.png', 'type': 'Industry talk', 'fields': [{'__typename': 'Core_SelectField'}, {'__typename': 'Core_MultipleSelectField', 'name': 'Language', 'values': []}, {'__typename': 'Core_MultipleSelectField', 'name': 'Business outcome(s)', 'values': [{'text': 'Create more sustainable processes and results'}, {'text': 'Design and manufacture better products'}, {'text': 'Increase operational efficiency'}]}, {'__typename': 'Core_MultipleSelectField', 'name': 'Product(s)', 'values': [{'text': 'Fusion 360'}]}, {'__typename': 'Core_MultipleSelectField', 'name': 'Industry', 'values': [{'text': 'Manufacturing Other'}]}, {'__typename': 'Core_MultipleSelectField', 'name': 'Audience Occupations', 'values': [{'text': 'Manufacturing Engineer'}, {'text': 'Mechanical Engineer'}, {'text': 'Product Engineer'}]}, {'__typen

Access only the classes (nodes) available.
Extract categories, attendees and class title

In [4]:
def get_fields(fields, name):
  for f in fields:
    if "name" in f and f["name"] == name:
      return [x["text"] for x in f["values"]]

records = []

for item in data:
  temp = item
  temp['category'] = None
  if item['categories']:
    temp['category'] = [c["name"] for c in item['categories']]
  temp['total_attendees'] = item['withEvent']['totalAttendees']
  temp['description'] = item['withEvent']['htmlDescription']
  temp['beginsAt'] = item['withEvent']['beginsAt']
  temp['class_name'] = item['withEvent']['title']

  temp['business_outcome'] = get_fields(item['fields'], "Business outcome(s)")
  temp['products'] = get_fields(item['fields'], "Product(s)")
  temp['industry'] = get_fields(item['fields'], "Industry")
  temp['audience_occupations'] = get_fields(item['fields'], "Audience Occupations")

  records.append(temp)

df_records = pd.DataFrame.from_records(records)
df_records.drop([ 'withEvent','categories', 'fields' ], axis=1, inplace=True)
print(f"{len(df_records)} classes found")
df_records

72 classes found


,bannerUrl,type,category,total_attendees,description,beginsAt,class_name,business_outcome,products,industry,audience_occupations
0,https://cdn-api.swapcard.com/public/images/4ec...,Industry talk,"[Additive Manufacturing, Machining, Sustainabi...",514,<p>Modern manufacturing technology has given d...,2022-09-27 08:00:00,MFG501099 | How Can Sustainable Manufacturing ...,[Create more sustainable processes and results...,[Fusion 360],[Manufacturing Other],"[Manufacturing Engineer, Mechanical Engineer, ..."
1,https://cdn-api.swapcard.com/public/images/83c...,Industry talk,"[Automation, Data Management, Digital Delivery...",1228,<p>This class will demonstrate the implementat...,2022-09-27 08:00:00,AS500850 | Autodesk Construction Cloud and Aut...,"[Better manage or reduce costs, Enable innovat...","[Autodesk BIM Collaborate Pro, BIM 360 Docs, F...",[Architecture],"[Architect, BIM Manager, Developer, Facilities..."
2,https://cdn-api.swapcard.com/public/images/eac...,Industry talk,"[Digital Transformation, Future of Work, Softw...",522,<p>The ability for teams to address and adapt ...,2022-09-27 08:00:00,CES501754 | Equipping Staff for the Digital Fu...,"[Enable innovation, Enhance creative excellenc...","[AutoCAD, BIM 360 Docs, Civil 3D, Revit]",[Civil Engineering],"[Education Administrator, Education Mentor, Te..."
3,https://cdn-api.swapcard.com/public/images/5a9...,Case study,"[Automation, Building Design, Building Informa...",183,<p>The New Government Quarter is an ambitious ...,2022-09-27 08:00:00,AS502169 | Automating the New Government Quart...,"[Better manage or reduce costs, Enable innovat...",[],[Architecture],"[Architect, BIM Manager, Developer, Facilities..."
4,https://cdn-api.swapcard.com/public/images/6af...,Technical instruction,"[Injection Molding, Product Design, Simulation]",257,<p>Designing parts for the injection molding p...,2022-09-27 08:00:00,CP500829 | How to Design Plastic Parts More Ef...,"[Design and manufacture better products, Incre...","[Fusion 360, Inventor]",[Consumer Products],"[Drafter, Mechanical Designer, Mechanical Deta..."
...,...,...,...,...,...,...,...,...,...,...,...
67,https://cdn-api.swapcard.com/public/images/e6b...,Industry talk,"[Artificial Intelligence and Machine Learning,...",1249,<p>This class will encompass creating a design...,2022-09-29 15:00:00,BES500957 | Design Technology Leadership: Inno...,"[Enable innovation, Gain general industry lear...",[BIM 360 Docs],[Building Engineering],"[Architect, BIM Manager, CAD Manager, Drafter,..."
68,https://cdn-api.swapcard.com/public/images/e9e...,Product demo,"[Digital Construction, Generative Design, Infr...",398,<p>This session will look at Grading Optimizat...,2022-09-29 15:00:00,CES501760 | Grading Optimization: Making Choic...,"[Better manage or reduce costs, Increase opera...",[Civil 3D],[Civil Engineering],"[Civil Engineer/Designer, Drafter, Environment..."
69,https://cdn-api.swapcard.com/public/images/0e1...,Case study,"[Cloud Collaboration, Digital Factory, Digital...",895,<p>Building projects are already realized with...,2022-09-29 16:30:00,MFG501919 | The Future of Integrated Factory M...,"[Better manage or reduce costs, Enable innovat...","[Autodesk BIM Collaborate Pro, Autodesk Docs, ...",[Manufacturing Other],"[BIM Manager, CAD Manager, Manufacturing Engin..."
70,https://cdn-api.swapcard.com/public/images/544...,Industry talk,"[Carbon Management, Data Management, Digital D...",504,"<p>In 2018, SYSTRA launched an R&D program cal...",2022-09-29 16:30:00,CES500878 | SYSTRA Digital Engineering Innovat...,"[Better manage or reduce costs, Create more su...",[Forge],[Civil Engineering],"[Business Development Representative, Owner/Op..."


# Top 10 classes by attendees

In [5]:
class_attendees = df_records[['class_name', 'total_attendees']]
top10_classes_by_attendees = class_attendees.sort_values(by='total_attendees',ascending=False)
top10_classes_by_attendees.head(10)

,class_name,total_attendees
12,674914 | General Session,3168
33,676706 | General Session Day 2,2798
41,678177 | Autodesk Construction Session,2690
56,CS501355 | Model Coordination or Navisworks fo...,1992
14,CI502227 | Giant Steps: Advances in Seamless B...,1957
19,AS502204 | Solving the Clash Between Model Coo...,1837
52,CS501621 | Microsoft Power BI to Maximize BIM ...,1755
30,CS501147 | Future of Work: How Do You Implemen...,1623
47,BLD501424 | Demystifying Digital Twins,1608
17,677160 | Forge Developer Session,1378


In [6]:
top10_classes_by_attendees.sort_values(by='total_attendees',ascending=True).plot(y="class_name", x="total_attendees",  kind="barh")


# Top 10 Categories by attendees

In [26]:
_= []
for i, r in df_records.iterrows():
  if r["category"]:
    for c in r["category"]:
      _.append(pd.Series({"class_name":r["class_name"],'total_attendees': r['total_attendees'], "category":c }))
      

class_attendees_per_categories = pd.DataFrame(_)
top10_classes = class_attendees_per_categories.groupby(['category'])['total_attendees'].sum().reset_index().sort_values(by='total_attendees',ascending=False).head(10)
top10_classes

,category,total_attendees
11,Building Information Modeling (BIM),22434
15,Cloud Collaboration,15480
25,Digital Transformation,12978
20,Data Management,11911
18,Construction Management,11431
26,Digital Twin,8599
23,Digital Delivery,8371
7,Automation,7917
13,CAD Management,7552
43,Model Coordination,7531


In [29]:
top10_classes.plot(x="category", y="total_attendees",  kind="bar", color="category")

# Top 10 talks by software




In [31]:
_= []
for i, r in df_records.iterrows():
  if r["products"]:
    for product in r["products"]:
      _.append(pd.Series({"class_name":r["class_name"],'total_attendees': r['total_attendees'], "products":product }))
      

class_attendes_per_software = pd.DataFrame(_)
top10_classes_by_soft = class_attendes_per_software.groupby(['products'])['total_attendees'].sum().reset_index().sort_values(by='total_attendees',ascending=False).head(10)
top10_classes_by_soft


,products,total_attendees
32,Revit,21702
3,AutoCAD,9351
11,Autodesk Docs,8706
7,Autodesk BIM Collaborate,8579
10,Autodesk Construction Cloud Connect,7499
17,Civil 3D,7306
21,Forge,6902
15,BIM 360 Docs,6767
8,Autodesk BIM Collaborate Pro,6683
9,Autodesk Build,6213


In [34]:
top10_classes_by_soft.plot(x="products", y="total_attendees",  kind="bar", color="products")

# Top 10 talks by Audience occupation

In [35]:
_= []
for i, r in df_records.iterrows():
  if r["audience_occupations"]:
    for product in r["audience_occupations"]:
      _.append(pd.Series({"class_name":r["class_name"],'total_attendees': r['total_attendees'], "audience_occupations":product }))
      

class_attendes_per_audience_occupations = pd.DataFrame(_)
top10_classes_by_occupations = class_attendes_per_audience_occupations.groupby(['audience_occupations'])['total_attendees'].sum().reset_index().sort_values(by='total_attendees',ascending=False).head(10)
top10_classes_by_occupations


,audience_occupations,total_attendees
6,BIM Manager,28743
59,VDC Manager,21548
58,Technology/Innovation Manager,20244
5,Architect,18824
41,Project Manager,16723
34,Owner/Operator,12395
57,Technical Director/Supervisor,9948
9,CAD Manager,9923
12,Developer,9423
40,Project Engineer,7543


In [36]:
top10_classes_by_occupations.plot(x="audience_occupations", y="total_attendees",  kind="bar", color="audience_occupations")

# Attendees per day and hour

In [13]:
df_records["day"] = pd.to_datetime(df_records["beginsAt"]).dt.day
class_attendees_per_categories= pd.DataFrame(columns=['class_name', 'total_attendees', 'category'])
for i, r in df_records.iterrows():
  if r["category"]:
    for c in r["category"]:
      n = {"class_name":r["class_name"],'total_attendees': r['total_attendees'], "category":c,"day":r["day"] }
      class_attendees_per_categories = class_attendees_per_categories.append(n,ignore_index=True)

class_attendees_per_categories_group = class_attendees_per_categories.groupby(['category','day'])['total_attendees'].sum().reset_index().sort_values(by='total_attendees',ascending=False)
class_attendees_per_categories_group

C:\Users\pdere\AppData\Local\Temp\ipykernel_16828\2155755722.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\pdere\AppData\Local\Temp\ipykernel_16828\2155755722.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\pdere\AppData\Local\Temp\ipykernel_16828\2155755722.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\pdere\AppData\Local\Temp\ipykernel_16828\2155755722.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\pdere\AppData\Local\Temp\ipykernel_16828\2155755722.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

,category,day,total_attendees
25,Building Information Modeling (BIM),29.0,9650
24,Building Information Modeling (BIM),28.0,8124
34,Cloud Collaboration,27.0,7864
47,Data Management,27.0,6258
36,Cloud Collaboration,29.0,5420
...,...,...,...
45,Convergence,28.0,156
123,Water Resource Management,27.0,148
77,Game Development,27.0,144
121,Virtual Production,27.0,144


In [14]:
class_attendees_per_categories_group.sort_values(by='day',ascending=True).plot(x="day", y="total_attendees",  kind="bar")


In [15]:
data_categories = class_attendees_per_categories.groupby(['day', 'category'])['day'].count().unstack('category').fillna(0)
# print(data.head())
data_categories.plot(kind='bar', color="category")
# data.plot(kind='bar', stacked=True)


In [16]:


print(hex_to_rgba("#B4FBB8", 0.5))

NameError: name 'hex_to_rgba' is not defined

In [ ]:
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

def hex_to_rgba(hex, alpha):
  h = hex.lstrip('#')
  rgb =  [int(h[i:i+2], 16) for i in (0, 2, 4)]
  rgb.append(alpha)
  return "rgba" +str(tuple(rgb))


df_records["day"] = pd.to_datetime(df_records["beginsAt"]).dt.day

software = [
    "revit",
            "navisworks",
            "forge",
            "autocad",
            "civil3d",
            "civil 3d",
            "formit",
            "infraworks",
            "dynamo",
            "fusion",
            "bim 360",
            # "unreal",
            "vault",
            "inventor",
            # "omniverse",
            # "ilogic",
            "construction cloud",
            "bim collaborate pro",
            # "vcad",
            # "arcgis",
            # "power bi"
             ]

class_attendes_per_software = pd.DataFrame(columns=['class_name', 'software','total_attendees', 'day'])
for i, r in df_records.iterrows():
  visited = False
  for s in software:
    if s in r["description"].lower() or s in r["class_name"].lower():
      n = {"class_name":r["class_name"],"total_attendees":r["total_attendees"], "software":s, 'day':str(r['day']) }
      class_attendes_per_software = class_attendes_per_software.append(n,ignore_index=True)
      visited=True
  # if not visited:
  #   n = {"class_name":r["class_name"],"total_attendees":r["total_attendees"], "software":"others", 'day':str(r['day']) }
  #   class_attendes_per_software = class_attendes_per_software.append(n,ignore_index=True)

class_attendes_per_software.sort_values(["day"])
opacity = "80"
nodes = np.unique(class_attendes_per_software[["software", "day"]], axis=None)
nodes = pd.Series(index=nodes, data=range(len(nodes)))
fig = go.Figure(
    go.Sankey(
        node={
  
            "thickness" : 15,
            "label": nodes.index, 
            # "color": [
            #     hex_to_rgba(px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)], 0.5) 
            #     for i in nodes
            # ]
            },
        link={
            "source": nodes.loc[class_attendes_per_software["software"]],
            "target": nodes.loc[class_attendes_per_software["day"]],
            "value": class_attendes_per_software["total_attendees"],
            "color": [
                hex_to_rgba(px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)], 0.5) 
                for i in nodes.loc[class_attendes_per_software["software"]]
            ],
        },
    )
)
fig.update_layout(title_text="Amount of attendees per day grouped by software",
                  font_size=10)




amount of attendees per hour per day

classes vs voted classes

Twitter /google trends vs AU 2022 day by day
